In [1]:
import matplotlib
matplotlib.use("Agg")

In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import os

Using TensorFlow backend.


In [3]:
IMG_HELP_PATH = '/Users/shankar/dev/code/ds/studies/data_science/dlcv'
import sys
sys.path.append(os.path.abspath(IMG_HELP_PATH))
from common.nn.conv import MiniVGGNet

In [4]:
OUTPUT = 'output'
MODELS = 'models'
NUM_MODELS = 3
EPOCHS = 2

In [5]:
# Load the training and testing data, then scale it into the range[0, 1]
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [6]:
# Convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [7]:
# Initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"]

In [8]:
# Construct  the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
    horizontal_flip=True, fill_mode="nearest")

In [9]:
# Loop over the number of models to train
for i in np.arange(0, NUM_MODELS):
    # Initialize the optimizer and model
    opt = SGD(lr=0.001, decay=0.01/EPOCHS, momentum=0.9, nesterov=True)
    model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    
    # Train the network
    H = model.fit_generator(aug.flow(trainX, trainY, batch_size=64), 
        validation_data=(testX, testY), epochs=EPOCHS, 
        steps_per_epoch=len(trainX)//64, verbose=1)
    
    # Save the model to disk
    p = [MODELS, "model_{}.model".format(i)]
    model.save(os.path.sep.join(p))
    
    # Evaluate the network
    predictions = model.predict(testX, batch_size=64)
    report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), 
        target_names=labelNames)
    
    # Save the classification reprot to file
    p = [OUTPUT, "model_{}.txt".format(i)]
    f = open(os.path.sep.join(p), "w")
    f.write(report)
    f.close()
    
    # Plot the training loss and accuracy
    p = [OUTPUT, "model_{}.png".format(i)]
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, EPOCHS), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, EPOCHS), H.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy for model {}".format(i))
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.savefig(os.path.sep.join(p))
    plt.close()

Epoch 1/2
781/781 [==============================] - 254s 326ms/step - loss: 2.2358 - acc: 0.3027 - val_loss: 1.5600 - val_acc: 0.4514
Epoch 2/2
781/781 [==============================] - 248s 318ms/step - loss: 1.8522 - acc: 0.3767 - val_loss: 1.4115 - val_acc: 0.4964
Epoch 1/2
781/781 [==============================] - 249s 319ms/step - loss: 2.2833 - acc: 0.2940 - val_loss: 1.4970 - val_acc: 0.4488
Epoch 2/2
781/781 [==============================] - 251s 321ms/step - loss: 1.9000 - acc: 0.3616 - val_loss: 1.4108 - val_acc: 0.4831
Epoch 1/2
781/781 [==============================] - 258s 330ms/step - loss: 2.2270 - acc: 0.3111 - val_loss: 1.4509 - val_acc: 0.4756
Epoch 2/2
781/781 [==============================] - 256s 328ms/step - loss: 1.8424 - acc: 0.3827 - val_loss: 1.3623 - val_acc: 0.5082
